In [3]:
import numpy as np
import pandas as pd
import os 
import logging
logging.getLogger().setLevel(logging.INFO)

# Filtering pyCBC Triggers

## Load Triggers

In [4]:
from pycbc_trigger_parser import TriggerTable, TriggerTableTimes, TemplateBank
from pycbc_trigger_parser.trigger_table_webpage import TriggerPage
from pycbc_trigger_parser.utils import filter_dataframe
from pycbc_trigger_parser.plotting import plot_template_bank
from pycbc_trigger_parser.data.o3 import chunk6

05:23 bilby INFO    : Running bilby version: 0.5.5: (CLEAN) 6415c788 2019-08-28 18:36:31 -0500


In [5]:
csv_file_path = "trigger_files/triggers.csv"
trigger_table = TriggerTable.from_csv(csv_file_path)
trigger_dataframe = trigger_table.to_dataframe()
trigger_dataframe.describe().transpose()

INFO:root:Generating trigger table from trigger_files/triggers.csv
INFO:numexpr.utils:Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


,count,mean,std,min,25%,50%,75%,max
ifar,382933.0000,1.2755,105.7602,0.0000,0.0000,0.0000,0.0031,51034.4521
stat,382933.0000,7.2276,0.9315,5.6569,6.4626,7.3286,8.0360,10.0297
snr,382933.0000,7.8521,0.8565,6.0703,7.2098,7.7000,8.3767,55.8182
timeslide_value,382933.0000,5378.4193,359264.8056,-741678.0000,-240240.0000,5670.0000,254940.0000,741840.0000
H1_snr,382933.0000,5.4968,0.6615,4.0064,5.0542,5.3283,5.8302,55.5747
H1_start_time,382933.0000,1242155569.5496,250441.3555,1241739046.0709,1241861923.3600,1242242403.1848,1242367991.8006,1242482422.4059
H1_template_duration,382933.0000,32.9629,29.0838,0.1501,9.6509,23.5294,51.8830,147.2004
H1_trigger_id,382933.0000,349129257.4608,201462962.7821,995.0000,174649880.0000,349505685.0000,523658614.0000,698164353.0000
L1_snr,382933.0000,5.5771,0.7952,4.0007,5.0636,5.3525,5.9272,29.3958
L1_start_time,382933.0000,1242150191.1303,249208.8890,1241739000.0301,1241860304.4521,1242237357.9151,1242363926.4895,1242482406.3820


## Filter Triggers based on Duration

In [6]:
MAX_DURATION = 0.454 # Based on pyCBC's template duration
filtered_trigger_table = trigger_table.get_filtered_table(
    H1_template_duration_max=MAX_DURATION,
    L1_template_duration_max=MAX_DURATION,
)

INFO:root:382933 triggers before filtering
INFO:root:Filtering with the condition: (H1_template_duration<=0.454)&(L1_template_duration<=0.454)
INFO:root:1564 triggers after filtering


In [7]:
filtered_trigger_table.to_dataframe().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ifar,1564.0000,0.9303,12.8121,0.0000,0.0001,0.0025,0.0053,347.1731
stat,1564.0000,7.8583,0.5368,5.6569,7.5998,8.0053,8.1028,9.4194
snr,1564.0000,8.1524,1.8312,6.3928,7.2409,7.6101,8.2887,37.9007
timeslide_value,1564.0000,25329.8850,355435.3947,-731700.0000,-180105.0000,24214.5000,283065.0000,715500.0000
H1_snr,1564.0000,5.5828,1.3792,4.2516,4.9960,5.2645,5.6964,37.5622
H1_start_time,1564.0000,1242159453.0722,250219.2071,1241739115.5549,1241862183.5371,1242245679.8758,1242361531.7858,1242481987.9476
H1_template_duration,1564.0000,0.3169,0.0936,0.1501,0.2357,0.3325,0.4007,0.4539
H1_trigger_id,1564.0000,380233774.8683,197237732.0700,765113.0000,207400365.0000,415028968.0000,547862872.5000,697481944.0000
L1_snr,1564.0000,5.8389,1.6285,4.3486,5.0071,5.2495,5.8765,29.3958
L1_start_time,1564.0000,1242134123.1873,252345.1544,1241739214.9125,1241852154.5671,1242216665.7218,1242352348.3601,1242481930.5784


## Save filtered trigger table and times

In [12]:
OUTDIR = './triggers_filtered_by_454ms/'
os.makedirs(OUTDIR, exist_ok=True)

# same trigger table
trigger_filename = os.path.join(OUTDIR, "triggers.csv")
filtered_trigger_table.to_csv(trigger_filename)

# time txt files for analysis
logging.info("Generating trigger times table")
trigger_time = TriggerTableTimes(filtered_trigger_table)
trigger_time.save_time_txt_files(txt_dir=OUTDIR)
trigger_time.to_csv(os.path.join(OUTDIR, "times.csv"))

# summary pages
logging.info("Generating trigger page for filtered data")
trigger_page = TriggerPage(
    webdir=OUTDIR,
    trigger_csv=trigger_filename,
    title="Filtered O3 Chunk6 Triggers",
    number_samples=None,
    bank_file_path='template_bank_masses_with_event.png',
    start_time=chunk6.START_TIME,
    end_time=chunk6.END_TIME
    )
trigger_page.render()

INFO:root:Saving trigger table to ./triggers_filtered_by_454ms/triggers.csv
INFO:root:Generating trigger times table
INFO:root:saving trigger time data in txt files
INFO:root:Generating trigger time dataframe
INFO:root:Generating trigger page for filtered data
INFO:root:./triggers_filtered_by_454ms/O3_Triggers is being created.
INFO:root:Loading triggers into csv from ./triggers_filtered_by_454ms/triggers.csv
INFO:root:Completed loading triggers from ./triggers_filtered_by_454ms/triggers.csv
INFO:root:[PAGE-PLOTTING]: Generating trigger search parameter table
INFO:root:[PAGE-PLOTTING]: Generating trigger times table
INFO:root:[PAGE-PLOTTING]: Generating trigger times plot
INFO:root:Plotting ./triggers_filtered_by_454ms/O3_Triggers/times.html
INFO:root:Converting GPS-->datetimes
INFO:root:Converting datetimes-->traces
INFO:root:Plotting datetimes
100%|██████████| 1564/1564 [00:14<00:00, 116.10it/s]
INFO:root:[PAGE-PLOTTING]: Copying bank plot
INFO:root:[PAGE-PLOTTING]: Generating mass s

/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates
/home/avi.vajpeyi/projects/bilby_report/bilby_report/templates


## Get Max-Min Mass Parameter Values from triggers

In [ ]:
filtered_trigger_dataframe = filtered_trigger_table.to_dataframe()
mass_column_headers = filtered_trigger_dataframe.columns[filtered_trigger_dataframe.columns.str.contains('mass')]
mass_statistics = filtered_trigger_dataframe[mass_column_headers].describe()
mass_statistics.transpose()[['min','max','mean','std']]



In [ ]:
constraints = ""
constraints_dict = {}
for header in mass_column_headers:
    constraints_dict.update({
        f"{header}_min": mass_statistics[header]['min'],
        f"{header}_max": mass_statistics[header]['max'],
    })
    constraints += f"{mass_statistics[header]['min']:.2f} <= {header} <= {mass_statistics[header]['max']:.2f}\n"
print(constraints)

## Plot new constraints on template bank

In [ ]:
@np.vectorize
def my_criteria(m1: float, m2: float, mc: float, q: float, M: float) -> int:
    """
    :param m1: mass1 val
    :param m2: mass2 val
    :param mc: chirp mass val
    :param q: mass ratio val
    :param M: total mass val
    :return: 1 if above parameters inside criteria (defined in function), otherwise 0
    """
    if (
            mass_statistics.mass_1['min'] <= m1 <= mass_statistics.mass_1['max'] and
            mass_statistics.mass_2['min'] <= m2 <= mass_statistics.mass_2['max'] and
            mass_statistics.mass_chirp['min'] <= mc <= mass_statistics.mass_chirp['max'] and
            mass_statistics.mass_ratio['min'] <= q <= mass_statistics.mass_ratio['max'] and
            mass_statistics.mass_total['min'] <= M <= mass_statistics.mass_total['max']
    ):
        return 1
    else:
        return 0

In [ ]:
def add_s190521g_to_plot(plt, ax1, ax2, handles, web_dir):
    event_settings = dict(
        c='k', 
        marker='*', 
        s=100, 
        label='S190521g'
    )
    ax1.scatter(
        chunk6.S190521g['mass_1'],
        chunk6.S190521g['mass_2'],
        **event_settings
    )
    ax2.scatter(
        chunk6.S190521g['mass_ratio'],
        chunk6.S190521g['mass_chirp'],
        **event_settings
    )
    event_patch = ax1.scatter([], [], label="S190521g", marker="*", color='k')
    handles = handles + [event_patch]
    ax1.legend(handles=handles, fontsize='large', markerscale=3)

    plt.savefig(os.path.join(web_dir, 'template_bank_masses_with_event.png'))
    return plt
    
    
def plot_bank_with_constraints(save_dir='.'):
    template_bank = TemplateBank.from_pycbc_hdf_bank_file(chunk6.BANK_FILE)
    plt, ax1, ax2, handles = plot_template_bank(
        m1=template_bank.mass_1,
        m2=template_bank.mass_2,
        mc=template_bank.mass_chirp,
        q=template_bank.mass_ratio,
        M=template_bank.mass_total,
        filtering_criteria=my_criteria,
        web_dir=save_dir)
    add_s190521g_to_plot(plt, ax1, ax2, handles, save_dir)
      

In [ ]:
plt = plot_bank_with_constraints()

## Number of templates after filtering

In [ ]:
def get_number_of_templates_in_constraints(filtering_constraints):
    template_bank = TemplateBank.from_pycbc_hdf_bank_file(chunk6.BANK_FILE)
    num_original = len(template_bank.to_dataframe())
    filtered_template_bank = template_bank.get_filtered_bank(**constraints_dict)
    num_filtered = len(filtered_template_bank.to_dataframe())
    return num_original, num_filtered
    
num_original, num_filtered = get_number_of_templates_in_constraints(constraints_dict)
print(f"Total #templates: {num_original}\n#templates in prior: {num_filtered}")

